In [110]:
import cx_Oracle
#from spellchecker import SpellChecker

# Créer un objet SpellChecker pour le français
#spell = SpellChecker(language='fr')

# Paramètres de connexion
username = "SYS"
password = "2000"
hostname = "localhost"
port = "1521"
sid = "xe"

# Création de la chaîne DSN
dsn = cx_Oracle.makedsn(hostname, port, sid=sid)

# Connexion à la base de données
try:
    connection = cx_Oracle.connect(username, password, dsn, mode=cx_Oracle.SYSDBA)
    print("Connexion réussie à la base de données Oracle")

    # Création d'un curseur
    cursor = connection.cursor()

    # Activer DBMS_OUTPUT
    cursor.callproc("dbms_output.enable")



    # Keyword original
    keyword = 'confiance'
 
    corrected_keyword = clean_sentence(keyword)
    print(corrected_keyword)
    cursor.execute("""
DECLARE

    c SYS_REFCURSOR;
    result_row VARCHAR2(4000);
BEGIN
    c := search_phrase('SOUSSOUSDomaineFormation', 'Motscles', :keyword);

    LOOP
        FETCH c INTO result_row;
        EXIT WHEN c%NOTFOUND;
        DBMS_OUTPUT.PUT_LINE(result_row);
    END LOOP;

    CLOSE c;
END;

    """, keyword=corrected_keyword)

    # Récupérer et afficher les lignes de DBMS_OUTPUT
    statusVar = cursor.var(cx_Oracle.NUMBER)
    lineVar = cursor.var(cx_Oracle.STRING)
    while True:
        cursor.callproc("dbms_output.get_line", (lineVar, statusVar))
        if statusVar.getvalue() != 0:
            break
        print(lineVar.getvalue())

    # Fermeture du curseur
    cursor.close()

except cx_Oracle.DatabaseError as e:
    print("Erreur lors de la connexion à la base de données", e)

finally:
    # Fermeture de la connexion
    if connection:
        connection.close()
        print("Connexion à la base de données fermée")


Connexion réussie à la base de données Oracle
confiance
confiance, soi, best
creer, climat, confiance, clients
bases, retrouver, confiance, memoire, full, digital, heures, avis
Connexion à la base de données fermée
